# Sample From Mesh Files & Load Proteins

In this notebook, point clouds are created from mesh files using poisson disk sampling.

In [4]:
%load_ext autoreload
%autoreload 2

import os

import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
from tqdm import tqdm

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
def load_mesh_and_sample_points(file_name, num_points=5000):
    # Default number of points as indicated in the thesis
    mesh = o3d.io.read_triangle_mesh(file_name)
    pc = mesh.sample_points_poisson_disk(num_points)
    return np.asarray(pc.points)

In [ ]:
# Load data and convert to point clouds
path = 'mesh_files_new'
count = 0
for idx, file in tqdm(enumerate(os.listdir(path))):
    file_path = os.path.join(path, file)
    if os.path.isfile(file_path):
        label = ""
        for s in file[:-3]:
            if s.isalpha():
                label += s
        point_cloud = load_mesh_and_sample_points(file_path)
        np.savez('point_clouds/' + label + str(count) + '.npz', objects=point_cloud, classes=label)
        count += 1

In [ ]:
# This step was only needed once
# Can be ignored
path = 'point_clouds'
objects = []
labels = []
for idx, file in tqdm(enumerate(os.listdir(path))):
    file_path = os.path.join(path, file)
    if os.path.isfile(file_path):
        load_table = np.load(file_path, allow_pickle=True)
        if len(load_table['objects'].shape) > 2:
            for i in range(load_table['objects'].shape[0]):
                objects.append(load_table['objects'][i])
                labels.append(load_table['classes'][i])
        else:
            objects.append(load_table['objects'])
            labels.append(load_table['classes'])

In [ ]:
np.savez('point_clouds/all_point_clouds', objects=objects, labels=labels)

In [ ]:
load_clouds = np.load('point_clouds/mc_gill_whole.npz', allow_pickle=True)
point_cloud_data = [load_clouds['objects'], load_clouds['labels']]

In [ ]:
def create_scatterplot_3d(data, descriptor):
    figure = plt.figure(figsize=(5, 5))
    ax = figure.add_subplot(111, projection='3d')

    # Map classes to colors
    unique_classes = data['class'].unique()
    colors = plt.cm.tab10(range(len(unique_classes)))
    class_color_map = dict(zip(unique_classes, colors))

    # Plot points by class
    for class_label in unique_classes:
        class_data = data[data['class'] == class_label]
        ax.scatter(
            class_data['evrap_x'], class_data['evrap_y'], class_data['evrap_z'],
            color=class_color_map[class_label], label=class_label, s=50
        )

    ax.legend()
    ax.view_init(80, 10)
    ax.set_title(descriptor)


def create_scatterplot_2d(data, descriptor):
    figure = plt.figure(figsize=(5, 5))
    ax = figure.add_subplot(111)

    # Map classes to colors
    unique_classes = data['class'].unique()
    colors = plt.cm.tab10(range(len(unique_classes)))
    class_color_map = dict(zip(unique_classes, colors))

    # Plot points by class
    for class_label in unique_classes:
        class_data = data[data['class'] == class_label]
        ax.scatter(
            class_data['samp_x'], class_data['samp_y'],
            color=class_color_map[class_label], label=class_label, s=50
        )

    ax.legend()
    ax.set_title(descriptor)


def create_scatterplot_1d(data, descriptor):
    figure = plt.figure(figsize=(5, 5))
    ax = figure.add_subplot(111)

    # Map classes to colors
    unique_classes = data['class'].unique()
    colors = plt.cm.tab10(range(len(unique_classes)))
    class_color_map = dict(zip(unique_classes, colors))

    # Plot points by class
    for class_label in unique_classes:
        class_data = data[data['class'] == class_label]
        x = np.arange(len(class_data[descriptor]))
        ax.scatter(
            x, class_data[descriptor], color=class_color_map[class_label], label=class_label,
        )

    ax.legend()
    ax.set_title(descriptor)

## Runtime Overview

The descriptors used have the following complexities. 

- EVARP: $O(nm)$
- SAMP: $O(nm)$
- SCOMP: $O(n \cdot (m$ log $m)$
- SIRM: $O(nm)$
- Shell Model: $O(nm)$
- Sector Model: $O(nm)$
- Combined Model: $O(n(m + m))$
- FPFH: $O(nmk)$

Notation:
- n: Number of point clouds
- m: Number of points per point cloud
- k: A hyperparameter of FPFH (relatively small), similar to k in k-NN

# Prepare Proteins

In [11]:
import requests
import warnings
import pandas as pd
from Bio.PDB import *    

In [12]:
# https://search.rcsb.org/index.html#building-search-request
# https://search.rcsb.org/index.html#search-example-4
# https://www.rcsb.org/docs/search-and-browse/advanced-search/structure-similarity-search
def fetch_cluster_ids(entry_id):
    url = 'https://search.rcsb.org/rcsbsearch/v2/query'

    query_dict = {
        "query": {
            "type": "terminal",
            "service": "structure",          # structural similarity is what we want
            "parameters": {
                "value": {
                    "entry_id": entry_id,
                    "assembly_id": "1"
                }
            }
        },
        "return_type": "entry",
        "request_options": {
            "paginate": {
                "start": 0,
                "rows": 100
            }
        }
    }

    query_response = requests.post(url, json=query_dict)
    results = query_response.json()
        
    candidate_cluster_ids = []
    for entry in results['result_set']:
        entry_id = entry['identifier']
        score = entry['score']
        
        # this value is arbitrary for now
        if score > 0.2:
            candidate_cluster_ids.append(entry_id)
    
    return candidate_cluster_ids

Problem: RCSB only has information about whether a protein is in its native confirmation in some cases, therefore we can not guarantee to build clusters of similar items. AlphaFold DB on the other hand has no tool to query similar proteins.

Generally, what do we want to cluster? Just get proteins in similar structure from the database, since they commonly also have similar functions? This creates a high bias, but is probably the only way.

Next: How similar should proteins be based on their similarity score?

Protein suggestions:

- 1A4U - Hemoglobin, a protein responsible for oxygen transport in the blood.
- 1GZX - Lysozyme, an enzyme that breaks down bacterial cell walls.
- 1UBQ - Ubiquitin, a small regulatory protein involved in protein degradation.
- 3MHT - DNA polymerase I, an enzyme that synthesizes DNA molecules.
- 4HHB - Myoglobin, a protein that stores oxygen in muscle tissue.
- 6VXX - SARS-CoV-2 spike glycoprotein, involved in viral entry into host cells.
- 2RH1 - Beta-2 adrenergic receptor, a G-protein coupled receptor (GPCR).
- 5XTL - Insulin receptor, important for glucose metabolism regulation.
- 3KZ8 - Cytochrome c oxidase, involved in the electron transport chain.
- 2C9T - Glutamate receptor, a ligand-gated ion channel in the nervous system.

- Note 4.01.2025: 1GZX deleted, since it has too many overlaps with myoglobin.

In [13]:
identifiers = ["1A4U", "1UBQ", "3MHT", "4HHB", "6VXX", "2RH1", "5XTL", "3KZ8", "2C9T"]

candidate_clusters = []
for protein_id in tqdm(identifiers):
    candidate_ids = fetch_cluster_ids(protein_id)
    candidate_clusters.append(candidate_ids)

100%|██████████| 9/9 [00:07<00:00,  1.16it/s]


In [14]:
labels = {
    'id': [],
    'label': []
}
identifier = 0
for cluster in candidate_clusters:
    print(len(cluster))
    for protein_id in cluster:
        labels['id'].append(protein_id)
        labels['label'].append(identifier)
    identifier += 1

9
44
27
100
43
19
67
23
1


In [7]:
labels_df = pd.DataFrame(labels)
labels_df.to_csv('point_clouds/proteins/labels.csv', index=False)

In [ ]:
def download_protein_by_id(entry_id):
    url = f"https://files.rcsb.org/download/{entry_id}.cif"
    protein_path = f'point_clouds/proteins/cif/{entry_id}.cif'

    if not os.path.isfile(protein_path):
        query_response = requests.get(url)
        if query_response.status_code == 200:
            with open(path, "wb") as f:
                f.write(query_response.content)

In [ ]:
for cluster in tqdm(candidate_clusters):
    for protein_id in cluster:
        download_protein_by_id(protein_id)

In [15]:
parser = MMCIFParser()

point_clouds_data = []
labels = []

warnings.filterwarnings("ignore")       # there are a lot of structural mistakes in PDB
for cluster in tqdm(candidate_clusters):
    for protein_id in cluster:
        structure = parser.get_structure("PHA-L", f"point_clouds/proteins/cif/{protein_id}.cif")
        coordinates = []
        for model in structure:
            for chain in model:
                for residue in chain:
                    for atom in residue:
                        # Get the atomic coordinates (x, y, z)
                        coord = atom.coord
                        coordinates.append(coord)
        if len(labels_df.loc[labels_df['id'] == protein_id, 'label']) > 1:
            continue
        point_clouds_data.append(np.array(coordinates))
        label = labels_df.loc[labels_df['id'] == protein_id, 'label'].iloc[0]
        labels.append(label)

100%|██████████| 9/9 [03:09<00:00, 21.06s/it]


In [18]:
len(labels)

244

In [16]:
import pickle

def save_proteins(point_clouds, clusters):
    pc_dict = {
        'objects': point_clouds,
        'labels': clusters
    }
    with open('point_clouds/proteins.pkl', 'wb') as f:
        pickle.dump(pc_dict, f)

In [17]:
save_proteins(point_clouds_data, labels)